In [1]:
import requests

from datetime import datetime
import pytz

import hmac
import hashlib

import base64

import urllib.parse

#TX
#https://pslcdatashop.web.cmu.edu/services/datasets/[id]/[?samples/id]/transactions

ACCESS_KEY_ID = 'MyAPP'
SECRET_ACCESS_KEY = 'MySECRET'

DATASET_ID = 5415
SAMPLE_ID = 7658

def _hmac_sha1(input_str):
    # Step 1 Create Hash
    raw = input_str.encode("utf-8")
    print(raw)
    key = SECRET_ACCESS_KEY.encode('utf-8')
    print(key)
    hashed = hmac.new(key, raw, hashlib.sha1)
    print(hashed)
    # Step 2 Create UTF-8 representation of hash
    hashed_digest = hashed.digest()
    print(hashed_digest)
    encoded = base64.encodebytes(hashed_digest)
    print(encoded)
    clean_utf8 = encoded.decode('utf-8')
    print(clean_utf8)
    #clean_utf8 = clean_utf8.replace('=\n', '') # need this step?
    # Step 3: Encode UTF-8 representation to URL
    clean_url = urllib.parse.urlencode({'': clean_utf8})[1:]
    print(clean_url)
    return clean_url

now_utc = datetime.now(pytz.utc)
http_date = now_utc.strftime('%a, %d %b %Y %H:%M:%S GMT')

STRING_TO_SIGN = f"""method: GET
contentMD5: 
contentType: 
timestamp: {http_date}
path: /datasets/{DATASET_ID}/samples/{SAMPLE_ID}/transactions"""

signature = _hmac_sha1(STRING_TO_SIGN)

print(http_date)

b'method: GET\ncontentMD5: \ncontentType: \ntimestamp: Sat, 13 May 2023 03:53:12 GMT\npath: /datasets/5415/samples/7658/transactions'
b'MySECRET'
b'z\xbb\x867\xde\x7f\x1b\xed\xd4]\x01\xef\xb5mk\x80\xc5T#\x1a'
b'eruGN95/G+3UXQHvtW1rgMVUIxo=\n'
eruGN95/G+3UXQHvtW1rgMVUIxo=

eruGN95%2FG%2B3UXQHvtW1rgMVUIxo%3D%0A
Sat, 13 May 2023 03:53:12 GMT


In [2]:
s = requests.Session()
s.headers.update({
    'date': http_date,
    'authorization': 'DATASHOP ' + ACCESS_KEY_ID + ':' + signature
})

URL = f"https://pslcdatashop.web.cmu.edu/services/datasets/{DATASET_ID}/samples/{SAMPLE_ID}/transactions"

PARAMS = {
    'cfs': 'all',
    'limit': '5000',
    'offset': '0'
}

req = requests.models.PreparedRequest()
req.prepare_url(URL, PARAMS)
response = s.get(req.url)

#steps
#http://pslcdatashop.web.cmu.edu/services/datasets/[id]/[?samples/id]/steps

# API Docs
# https://pslcdatashop.web.cmu.edu/api/

# Analyses can also be automated with LearnSphere
# http://learnsphere.org/help.html
# https://www.youtube.com/embed/GmW3brm6_0c

In [3]:
print(STRING_TO_SIGN)

method: GET
contentMD5: 
contentType: 
timestamp: Sat, 13 May 2023 03:53:12 GMT
path: /datasets/5415/samples/7658/transactions


In [4]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\n<pslc_datashop_message result_code="-100" result_message="Error. Authorization failed.  Check your credentials."/>\n'